In [ ]:
import glob

import essentia
import librosa
import librosa.display
import matplotlib.pyplot as plt
import mir_eval
import numpy as np
import pandas as pd
# from tdqm import tdqm

import IPython.display as ipd
plt.rcParams["figure.figsize"] = (15,5)

In [ ]:
def plot_comparison(x, fs, expected_beats, librosa_beats, start=0, end=50):
    fig, ax = plt.subplots(nrows=1, sharex=True)
    ax.plot(x[start*fs:end*fs], label='waveform')

    ax.vlines(
        (expected_beats[(expected_beats >= start) & (expected_beats <= end)]-start)*fs, 
              0, 1, alpha=0.5, color='r', linestyle='--', label='groundtruth'
    )
    ax.vlines(
        (librosa_beats[(librosa_beats >= start) & (librosa_beats <= end)]-start)*fs, 
        0, 1, alpha=0.5, color='g', linestyle='--', label='librosa')

    ax.legend()

In [ ]:
# CONSTANTS
FS = 44100

In [ ]:
# dataset available in https://www.eumus.edu.uy/candombe/datasets/ISMIR2015/dataset.html
file_path = [i[:-4] for i in glob.glob('../datasets/candombe/*.wav')]

In [ ]:
dataset_result = {}

for file in file_path: #tqdm (file_path, desc="Loading..."):
    print(f"processing {file}")
    x, fs = librosa.load(f"{file}.wav", mono=True, sr=FS)
    x_df = pd.read_csv(f"{file}.csv", names=["timestamp", "beat"])

    # calculate the beats using librosa approach
    _, beat_frame = librosa.beat.beat_track(x, FS)
    librosa_timestamps = librosa.frames_to_time(beat_frame, FS)

    # keep the notation the same as the dataset
    librosa_beats = [(i%4+1)/10 + (i//4) for i in range(len(librosa_timestamps))]

    librosa_df = pd.DataFrame({"librosa_timestamp": librosa_timestamps, "librosa_beat": librosa_beats})
    results_df = x_df.join(librosa_df)

    results_df['diff'] = results_df['timestamp'] - results_df["librosa_timestamp"]

    # if the difference between groundtruth and algorithm anotation are < 50ms, we should consider it correct 
    results_df['correct'] = results_df['diff'] < 0.5
    results_df.dropna(inplace=True)

    # pull all the results together in one place
    dataset_result[file]= {
        #"results_df": results_df, 
        #"stats": results_df["diff"].describe(), 
        #"mean_diff": results_df["diff"].describe()["mean"],
        #"max_diff": results_df["diff"].describe()["max"],
        #"std_diff": results_df["diff"].describe()["std"],
        #"median_diff": results_df["diff"].median(),
        "ground_truth_beats": results_df['timestamp'].values,
        "librosa_beats": results_df['librosa_timestamp'].values,
        "f_score": mir_eval.beat.f_measure(results_df['timestamp'].values, results_df['librosa_timestamp'].values),
        "p_score": mir_eval.beat.p_score(results_df['timestamp'].values, results_df['librosa_timestamp'].values)
    }

In [ ]:
# ordering from worse to best f-score
dataset_result = {k: v for k, v in sorted(dataset_result.items(), key=lambda item: item[1]["f_score"])}

In [ ]:
for i in dataset_result.items():
    print(f'{i[0]} -> {i[1]["f_score"]}')

In [ ]:
x, fs = librosa.load("../datasets/candombe/csic.1995_ansina2_04.wav", mono=True, sr=FS)

In [ ]:
tmp = dataset_result["../datasets/candombe/csic.1995_ansina2_04"]

In [ ]:
clicks_truth = mir_eval.sonify.clicks(tmp["ground_truth_beats"], FS, click=None, length=len(x))

wrong_click = np.sin(2*np.pi*np.arange(FS*.1)*500/(1.*FS))
# Exponential decay
wrong_click *= np.exp(-np.arange(FS*.1)/(FS*.01))

wrong_clicks = mir_eval.sonify.clicks(tmp["librosa_beats"], FS, click=wrong_click, length=len(x))

In [ ]:
ipd.Audio(x+clicks_truth+wrong_clicks, rate=FS)

In [ ]:
ipd.Audio(x+clicks_truth, rate=FS)

In [ ]:
ipd.Audio(x+wrong_clicks, rate=FS)

In [ ]:
# TODO: montar uma visualização que seja mostrada na função novidade em vez de na forma de onda
# TODO: fazer a mesma comparação utilizando a função de beat tracking do essentia 